In [ ]:
#default_exp web.demo

# Web Demo

> Simple demo UI.

# TODO

- add journal download option

In [ ]:
#export
from vulcan_medication_bundle.per_patient import *

from flask import Blueprint
from flask import flash
from flask import g
from flask import redirect
from flask import render_template
from flask import request
from flask import url_for
from flask import send_from_directory
from werkzeug.exceptions import abort
from werkzeug.utils import secure_filename
import tempfile,time,json
from pathlib import Path

bp = Blueprint("demo", __name__)

In [ ]:
#export
def get_single_patient_medication_bundle(api_base, subject):
    bundle = create_single_patient_medication_bundle(api_base, subject)
    bundle = handle_entry_search(bundle)
    bundle = filter_bundle(bundle, medication_status_filter)
    bundle = filter_bundle(bundle, do_not_perform_filter)
    return bundle

In [ ]:
#export
@bp.route("/", methods=("GET", "POST"))
def index():
    """Show all the index page."""
    medication_bundle = ''
    if request.method == "POST":
        api_base = request.form["api_base"]
        subject = request.form["subject"]
        print(api_base, subject)
        error = None

        if not api_base:
            error = "api_base is required."
        if not subject:
            error = "subject is required"

        if error is not None:
            flash(error)
        else:
            medication_bundle = get_single_patient_medication_bundle(api_base, subject)
            medication_bundle = json.dumps(medication_bundle, indent=2)
    
    return render_template("index.html", medication_bundle=medication_bundle)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_per_patient.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_demo.ipynb.
Converted index.ipynb.
